In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kgr9wovk
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kgr9wovk
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=fc9fa03d8b7b71efcc2e9919c44aef16c396bcd8bdc7dd324c1b5fa70351ffe7
  Stored in directory: /tmp/pip-ephem-wheel-cache-_uv60o1f/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu
#include <stdio.h>

#define N 1024 // Matrix size (N x N)

// CUDA kernel for matrix multiplication
__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int sum = 0;
    for (int i = 0; i < N; i++) {
        sum += a[row * N + i] * b[i * N + col];
    }

    c[row * N + col] = sum;
}

int main() {
    int *a, *b, *c;           // Host matrices
    int *d_a, *d_b, *d_c;     // Device matrices

    // Allocate memory for host matrices
    a = (int*)malloc(N * N * sizeof(int));
    b = (int*)malloc(N * N * sizeof(int));
    c = (int*)malloc(N * N * sizeof(int));

    // Initialize host matrices
    for (int i = 0; i < N * N; i++) {
        a[i] = i;
        b[i] = i;
    }

    // Allocate memory for device matrices
    cudaMalloc((void**)&d_a, N * N * sizeof(int));
    cudaMalloc((void**)&d_b, N * N * sizeof(int));
    cudaMalloc((void**)&d_c, N * N * sizeof(int));

    // Copy host matrices to device
    cudaMemcpy(d_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Set grid and block dimensions
    dim3 dimBlock(16, 16);
    dim3 dimGrid(N / dimBlock.x, N / dimBlock.y);

    // Launch the matrix multiplication kernel
    matrixMul<<<dimGrid, dimBlock>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print a few elements of the result matrix
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            printf("%d\t", c[i * N + j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}


894959616	895483392	896007168	896530944	897054720	897578496	898102272	898626048	899149824	899673600	
358088704	359661056	361233408	362805760	364378112	365950464	367522816	369095168	370667520	372239872	
-178782208	-176161280	-173540352	-170919424	-168298496	-165677568	-163056640	-160435712	-157814784	-155193856	
-715653120	-711983616	-708314112	-704644608	-700975104	-697305600	-693636096	-689966592	-686297088	-682627584	
-1252524032	-1247805952	-1243087872	-1238369792	-1233651712	-1228933632	-1224215552	-1219497472	-1214779392	-1210061312	
-1789394944	-1783628288	-1777861632	-1772094976	-1766328320	-1760561664	-1754795008	-1749028352	-1743261696	-1737495040	
1968701440	1975516672	1982331904	1989147136	1995962368	2002777600	2009592832	2016408064	2023223296	2030038528	
1431830528	1439694336	1447558144	1455421952	1463285760	1471149568	1479013376	1486877184	1494740992	1502604800	
894959616	903872000	912784384	921696768	930609152	939521536	948433920	957346304	966258688	975171072	
358088704	3